# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP NUMBER: **12** - please add your group number into the file name

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|Antonio Ramos |20240561|
|Tahiya Jahan Laboni|20240943|
|Joao Cardoso|20240529|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### Groups  

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### Submission      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/PATH/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/PATH/Neo4Jplugins`.
- Change the code below accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "c:\PATH\Neo4JPlugins":/plugins -v "c:\PATH\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

If the neo4j browser fails to load gds plugins, run the following in the Command Prompt before creating the container again:
`// Remove stopped containers //
docker container prune -f
// Remove unused images //
docker image prune -a -f
// Remove unused volumes //
docker volume prune -f
// Remove unused networks //
docker network prune -f
// Remove all unused resources in one command //
docker system prune -a -f`

In [31]:
from neo4j import GraphDatabase
from pprint import pprint

In [32]:
from neo4j import GraphDatabase
from pprint import pprint

# Connection details
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "test"

# Create Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [33]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

In [34]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

In [35]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

<span style="color:salmon"> 1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]<span>

`Step 1: Schema`

In [38]:
query = """
        CALL db.schema.visualization();
    """

result = execute_read(driver, query)

pprint(result)

[<Record nodes=[<Node element_id='-33' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-32' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-29' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-31' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-34' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-30' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-35' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>] relationships=[<Relationship element_id='-33' nodes=(<Node element_id='-32' labels=frozenset({'BEERS'}) properties={

`Step 2: check all node labels `

In [40]:
query = """
        CALL db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


`Step 3: Check all relationship types`

In [42]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


`Step 4: Check all property keys`

In [44]:
# Check all property keys
query = """
        CALL db.propertyKeys()
    """

result = execute_read(driver, query)

pprint(result)

[<Record propertyKey='name'>,
 <Record propertyKey='types'>,
 <Record propertyKey='notes'>,
 <Record propertyKey='state'>,
 <Record propertyKey='id'>,
 <Record propertyKey='abv'>,
 <Record propertyKey='retired'>,
 <Record propertyKey='availability'>,
 <Record propertyKey='brewery_id'>,
 <Record propertyKey='date'>,
 <Record propertyKey='score'>,
 <Record propertyKey='taste'>,
 <Record propertyKey='feel'>,
 <Record propertyKey='overall'>,
 <Record propertyKey='beer_id'>,
 <Record propertyKey='text'>,
 <Record propertyKey='smell'>,
 <Record propertyKey='look'>,
 <Record propertyKey='ABV'>]


`Step 5: check total number of nodes in the database`

In [46]:
# check total number of nodes in the database
query = """
        MATCH (n) RETURN COUNT(n) AS Total_Nodes
      """
result = execute_read(driver, query)
pprint(result)

[<Record Total_Nodes=3141348>]


`Step 6: check total number of relationships in the database`

In [48]:
# check total number of relationships in the database
query = """
        MATCH (n)-[r]->() RETURN COUNT(r) AS Total_Relationships
      """
result = execute_read(driver, query)
pprint(result)

[<Record Total_Relationships=6332824>]



<span style="color:salmon">2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
- We have to work with:
  1. Duplicate Beer records
  2. Beers not linked to breweries
  3. Redundand or missing relationships
  4. All adjustments here intend to improve database consistency<span>

`Step 2.1 : Removing nodes without relationships`

In [51]:
#counting

query = """
MATCH(c)
WHERE NOT EXISTS ((c)-[]-())
RETURN COUNT(c)
"""

result = execute_read(driver, query)

pprint(result)

[<Record COUNT(c)=0>]


In [52]:
#dropping

query = """
MATCH(c)
WHERE NOT EXISTS ((c)-[]-())
DETACH DELETE c
"""

result = execute_write(driver, query)

pprint(result)

[]


`Step 2.2: Fixing possible Beers Not Linked to Breweries`

In [54]:
#Total Beers

query = """
        MATCH (b:BEERS)
        RETURN COUNT(b)
    """

result = execute_read(driver, query)

pprint(result)

[<Record COUNT(b)=417746>]


In [55]:
#Beers Not Linked to Breweries count

query = """
        MATCH (b:BEERS)
        WHERE NOT EXISTS((b)<-[:BREWED]-(:BREWERIES))
        RETURN COUNT(b)
    """

result = execute_read(driver, query)

pprint(result)

[<Record COUNT(b)=0>]


In [56]:
#Beers Not Linked to Breweries, but linked to anything else 

query = """
        MATCH (b:BEERS)
        WHERE NOT (b)<-[:BREWED]-(:BREWERIES)
        AND EXISTS ((b)-[]-())
        RETURN count(b)
    """

result = execute_read(driver, query)

pprint(result)

#nothing to do

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: Coercion of list to boolean is deprecated. Please consider using `NOT isEmpty(...)` instead.} {position: line: 3, column: 27, offset: 43} for query: '\n        MATCH (b:BEERS)\n        WHERE NOT (b)<-[:BREWED]-(:BREWERIES)\n        AND EXISTS ((b)-[]-())\n        RETURN count(b)\n    '


[<Record count(b)=0>]


`Step 2.3: Duplicates`

In [58]:
#CITIES count

query = """
MATCH (c:CITIES)
RETURN count(c)
    """

result = execute_read(driver, query)

pprint(result)

[<Record count(c)=11665>]


In [59]:
#CITIES Duplicates

query = """
MATCH (c:CITIES)
WITH c.name AS name, COUNT(c) AS count
WHERE count > 1
RETURN count(name)
    """

result = execute_read(driver, query)

pprint(result)

#nothing to do

[<Record count(name)=0>]


In [60]:
#COUNTRIES count

query = """
MATCH (c:COUNTRIES)
RETURN count(c)
    """

result = execute_read(driver, query)

pprint(result)

[<Record count(c)=200>]


In [61]:
#COUNTRIES Duplicates

query = """
MATCH (c:COUNTRIES)
WITH c.name AS name, COUNT(c) AS count
WHERE count > 1
RETURN count(name)
    """

result = execute_read(driver, query)

pprint(result)

#nothing to do

[<Record count(name)=0>]


`Step 2.4: Finding Duplicate Breweries`

In [63]:
#total names duplicated

query = """
MATCH (c:BREWERIES)
WITH c.name AS name, COUNT(c) AS count
WHERE count > 1
RETURN count(name)
"""

result = execute_read(driver, query)
pprint(result)

[<Record count(name)=17909>]


* cell below to merge brewiries with the same name

`Step 2.5: Finding duplicate beer names`

In [66]:
#total names duplicated

query = """
MATCH (c:BEERS)
WITH c.name AS name, COUNT(c) AS count
WHERE count > 1
RETURN count(name)
"""

result = execute_read(driver, query)
pprint(result)

[<Record count(name)=65093>]


`Step 2.5: Finding Beers with invalid ABV values`

In [68]:
query = """
    MATCH (br:BEERS)
    WHERE br.ABV IS NULL OR br.ABV <= 0 OR toLower(br.ABV) = "nan"
    RETURN COUNT(br)
"""
result = execute_read(driver, query)
pprint(result)

[<Record COUNT(br)=417746>]


In [69]:
#we need to delete or update ABV values. This ABVs will be set to 0

query = """
    MATCH (br:BEERS)
    WHERE br.ABV IS NULL OR br.ABV <= 0 OR toLower(br.ABV) = "nan"
    SET br.ABV = "0"
"""

result = execute_write(driver, query)
pprint(result)

[]


In [70]:
#setting ABV to float (Question 4a)

query = """
    MATCH (br:BEERS)
    set br.ABV = toFloat(br.ABV)
"""

result = execute_write(driver, query)
pprint(result)

[]


<span style="color:salmon">Question 3: Analytics department requires the following information for the biweekly reporting:
1. How many reviews has the beer with the most reviews?
2. Which three users wrote the most reviews about beers?
3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
4. Which top three breweries produce the largest variety of beer styles?
5. Which country produces the most beer styles?</span>


`Question 1: How many reviews has the beer with the most reviews?`

In [73]:
# Question 1: How many reviews has the beer with the most reviews?
query = """
    MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
    RETURN b.name AS Beer_Name, COUNT(r) AS Total_Reviews
    ORDER BY Total_Reviews DESC
    LIMIT 1
"""

result = execute_read(driver, query)

pprint(result)


[<Record Beer_Name='IPA' Total_Reviews=8771>]


`Question 2: Which three users wrote the most reviews about beers?`

In [75]:
# Question 2: Which three users wrote the most reviews about beers?
query = """
    MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)
    RETURN u.name AS User, COUNT(r) AS Total_Reviews
    ORDER BY Total_Reviews DESC
    LIMIT 3;

"""

result = execute_read(driver, query)

pprint(result)


[<Record User='Sammy' Total_Reviews=3756>,
 <Record User='acurtis' Total_Reviews=3403>,
 <Record User='kylehay2004' Total_Reviews=3368>]


`Question 3: Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.`

In [77]:
# Question 3: Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
query = """
    MATCH (b:BEERS)
    Where toLower(b.notes) CONTAINS 'fruit'
        OR toLower(b.notes) CONTAINS 'complex'
        OR toLower(b.notes) CONTAINS 'nutty'
        OR toLower(b.notes) CONTAINS 'dark'
    RETURN b.name AS Beer, b.notes AS Description
    LIMIT 50

"""

result = execute_read(driver, query)

pprint(result)


[<Record Beer='Hefeweizen' Description='A pale, spicy, fruity, refreshing Hefeweizen originating in Southern Germany. The fast-maturing beer is lightly hopped with hallertau and shows a unique banana-and-clove yeast character. This is a specialty for summer consumption but enjoyed year-round by all. Prost!'>,
 <Record Beer='Wheat Ale' Description='A hefe-weizen with exotic top notes to the aromas. Predominantly banana with floral edges. On the palate the fruit notes continue with some caramel coming through on the rich elegant finish. Retains a good mousse throughout.'>,
 <Record Beer='Philadelphia Porter' Description='Originally enjoyed by the working class of England, porters inevitably became a staple in American brew pubs. Our interpretation is brewed with Caramel malt for a rich, toffee-like sweetness and a touch of East Kent Goldings hops. However, it’s the robust flavor of the black and chocolate malt that take center stage in this dark, full-bodied ale.'>,
 <Record Beer='Black 

`Question 4: Which top three breweries produce the largest variety of beer styles?`

In [79]:
# Question 4: Which top three breweries produce the largest variety of beer styles?
query = """
        MATCH (bre:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        RETURN bre.name AS Brewery, count(DISTINCT s.name) AS Total_Style
        ORDER BY Total_Style DESC
        LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)

[<Record Brewery='Iron Hill Brewery & Restaurant' Total_Style=94>,
 <Record Brewery='Rock Bottom Restaurant & Brewery' Total_Style=93>,
 <Record Brewery='Goose Island Beer Co.' Total_Style=88>]


`Question 5: Which country produces the most beer styles?`

In [81]:
# Question 5: Which country produces the most beer styles?
query = """
        MATCH (country:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLE)
        RETURN country.name AS Country, COUNT(DISTINCT s.name) AS Total_style
        ORDER BY Total_style DESC
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record Country='US' Total_style=113>]


<span style="color:salmon">4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]

1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.</span>


`Question 4(A): Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.`

In [84]:
query = """
        PROFILE
        MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
        WHERE b.abv IS NOT NULL AND b.abv <> "" 
          AND toFloat(b.abv) >= 0.5 AND toFloat(b.abv) <= 70  // Excludes non-alcoholic & extreme error entries in the database
        RETURN DISTINCT b.name AS Beer_Name, toFloat(b.abv) AS abv_score, br.name AS Brewery
        ORDER BY abv_score DESC
        LIMIT 5;
        
    """

result = execute_read(driver, query)

pprint(result)

[<Record Beer_Name='Snake Venom' abv_score=67.5 Brewery='Brewmeister'>,
 <Record Beer_Name='series 3' abv_score=66.0 Brewery='Redline Brewhouse'>,
 <Record Beer_Name='Armageddon' abv_score=65.0 Brewery='Brewmeister'>,
 <Record Beer_Name='Start The Future' abv_score=60.0 Brewery="Brouwerij 't Koelschip">,
 <Record Beer_Name='Schorschbräu Schorschbock 57%' abv_score=57.5 Brewery='Schorschbräu'>]


`Question 4(B): Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.`

In [86]:
# first, third most productive user
# second, find the top 5 distict beer styles based on highest average score (smell+feel) 

# step 1: Identify the third most productive user
query = """
        MATCH (r:REVIEWS)-[:POSTED]->(u:USER)
        RETURN u.name AS ThirdMostProductiveUser, COUNT(r) AS TotalReviews
        ORDER BY TotalReviews DESC
        SKIP 2
        LIMIT 1

    """

result = execute_read(driver, query)

pprint(result)



[<Record ThirdMostProductiveUser='kylehay2004' TotalReviews=3368>]


In [87]:
# third most productive user as "kylehay2004", with 3368 reviews.

# step 2: Let's use this user to find the top 5 distict beer styles with the highest average score of smell + feel
query = """
        PROFILE
        MATCH (u:USER {name:'kylehay2004'})<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WHERE toFloat(r.smell) IS NOT NULL AND toFloat(r.feel) IS NOT NULL
        WITH s.name AS Beer_style, AVG(toFloat(r.smell) + toFloat(r.feel)) AS Average_score
        RETURN DISTINCT Beer_style, Average_score
        ORDER BY Average_score DESC
        LIMIT 5;

    """

result = execute_read(driver, query)

pprint(result)



[<Record Beer_style='New England IPA' Average_score=8.654761904761903>,
 <Record Beer_style='American Imperial Stout' Average_score=8.546363636363628>,
 <Record Beer_style='Bière de Champagne / Bière Brut' Average_score=8.5>,
 <Record Beer_style='Belgian Fruit Lambic' Average_score=8.444444444444445>,
 <Record Beer_style='Belgian Gueuze' Average_score=8.42857142857143>]


<span style="color:salmon"> 5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]

`Question 5(B). Which beer is the most popular when considering the number of users who reviewed it?`

In [90]:
# Frist, find the number of unique users who reviewed each beer.
# second, sort the most popular of unique reviewers
query = """
        MATCH (b:BEERS)-[:REVIEWED]-()-->(u:USER)
        RETURN b.name AS beer_name, COUNT(DISTINCT u) AS reviews
        ORDER BY reviews DESC
        LIMIT 1;
        
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='IPA' reviews=6884>]


In [91]:
# Step 0 - Clear graph, graph names need to be unique
try:
    query = """
            CALL gds.graph.drop('most_popular_beer') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `most_popular_beer` does not exist on database `neo4j`. It might exist on another database.')


In [92]:
# Step 1 - Create an appropriate graph (a subgraph)
try:
    query = """
        CALL gds.graph.project.cypher(
            'most_popular_beer',
            
            'MATCH (n) WHERE n:USER OR n:BEERS RETURN id(n) AS id',
            
            'MATCH (b:BEERS)-[]->(r:REVIEWS)-[]->(u:USER) 
            RETURN id(b) AS source, id(r) AS weight, id(u) AS target'
            );
      """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)
    
#     First we find all the nodes of type User, and return only id values. Use limit while checking!
#     Second we look up the nodes and their relationships

[<Record nodeQuery='MATCH (n) WHERE n:USER OR n:BEERS RETURN id(n) AS id' relationshipQuery='MATCH (b:BEERS)-[]->(r:REVIEWS)-[]->(u:USER) \n            RETURN id(b) AS source, id(r) AS weight, id(u) AS target' graphName='most_popular_beer' nodeCount=524391 relationshipCount=2537991 projectMillis=10636>]


In [93]:
# Step 1 - part 1
try:
    query = """
            MATCH (n) WHERE n:USER OR n:BEERS RETURN id(n) AS id
            LIMIT 5
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record id=62212>,
 <Record id=62213>,
 <Record id=62214>,
 <Record id=62215>,
 <Record id=62216>]


In [94]:
# Step 1 - part 2
try:
    query = """
            MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[]->(u:USER) 
            RETURN id(b) AS source, id(r) AS weight, id(u) AS target
            LIMIT 15
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record source=391237 weight=590677 target=9494326>,
 <Record source=383463 weight=9486825 target=9494326>,
 <Record source=381076 weight=9461714 target=9494326>,
 <Record source=383077 weight=9470942 target=9494326>,
 <Record source=382144 weight=9450483 target=9494326>,
 <Record source=381175 weight=9438979 target=9494326>,
 <Record source=380254 weight=9411144 target=9494326>,
 <Record source=380355 weight=9418405 target=9494326>,
 <Record source=380240 weight=9414784 target=9494326>,
 <Record source=378010 weight=9379568 target=9494326>,
 <Record source=377230 weight=9354326 target=9494326>,
 <Record source=376945 weight=9356927 target=9494326>,
 <Record source=376609 weight=9342994 target=9494326>,
 <Record source=376581 weight=9343498 target=9494326>,
 <Record source=375278 weight=9328626 target=9494326>]


In [95]:
# Step 2 - Run the algorithm

try:
    query = """
        CALL gds.pageRank.stream('most_popular_beer', {relationshipWeightProperty:'weight'})
            
            YIELD nodeId, score
            
            RETURN gds.util.asNode(nodeId).name AS name, score
            
            ORDER BY score desc
            LIMIT 10
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)
    
# yield: In the stream execution mode, the algorithm returns the score for each node. 
# This allows us to inspect the results directly or post-process them in Cypher without any side effects.    
# return: retrieves username based on node ID, and the calculated score

[<Record name='Sammy' score=142.77564254316593>,
 <Record name='kylehay2004' score=113.24955466651426>,
 <Record name='StonedTrippin' score=99.96875878674035>,
 <Record name='Boone757' score=99.79166853566377>,
 <Record name='jaydoc' score=95.27986013569483>,
 <Record name='biboergosum' score=86.12672689371412>,
 <Record name='UCLABrewN84' score=85.0241176258547>,
 <Record name='Gagnonsux' score=81.38276174413348>,
 <Record name='LXIXME' score=79.01240243193882>,
 <Record name='Taphouse_Traveler' score=76.4680029435584>]


`Question 5(D). Which user is the most influential when it comes to reviews of distinct beers by style?`

In [97]:
query = """
        MATCH (u:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        RETURN u.name AS username, COUNT(DISTINCT s) AS distinct_styles_reviewed
        ORDER BY distinct_styles_reviewed DESC
        LIMIT 1;
    """

result = execute_read(driver, query)

pprint(result)

[<Record username='biboergosum' distinct_styles_reviewed=110>]


In [98]:
# Step 0 - Clear graph, graph names need to be unique

try:
    query = """
            CALL gds.graph.drop('top_influential_reviewers') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `top_influential_reviewers` does not exist on database `neo4j`. It might exist on another database.')


In [99]:
# Step 1 - Create an appropriate graph (a subgraph)

try:
    query = """
        CALL gds.graph.project.cypher(
            'top_influential_reviewers',
            
            'MATCH (n:USER) RETURN id(n) AS id
     UNION
     MATCH (n:STYLE) RETURN id(n) AS id',
            
            'MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
              RETURN id(s) AS source, COUNT(DISTINCT r) AS weight, id(u) AS target'
            )
      """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)
    
#     First we find all the nodes of type User, and return only id values. Use limit while checking!
#     Second we look up the nodes and their relationships

[<Record nodeQuery='MATCH (n:USER) RETURN id(n) AS id\n     UNION\n     MATCH (n:STYLE) RETURN id(n) AS id' relationshipQuery='MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)\n              RETURN id(s) AS source, COUNT(DISTINCT r) AS weight, id(u) AS target' graphName='top_influential_reviewers' nodeCount=106758 relationshipCount=893394 projectMillis=12210>]


In [100]:
# Step 1 - part 1
try:
    query = """
            MATCH (n) WHERE n:USER OR n:BEERS OR n:STYLE RETURN id(n) AS id
            LIMIT 5     
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record id=62212>,
 <Record id=62213>,
 <Record id=62214>,
 <Record id=62215>,
 <Record id=62216>]


In [101]:
# Step 1 - part 2
try:
    query = """
            MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
              RETURN id(u) AS source, COUNT(DISTINCT r) AS weight, id(s) AS target
            LIMIT 15
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record source=9496328 weight=5 target=9494213>,
 <Record source=9497034 weight=2 target=9494213>,
 <Record source=9498009 weight=3 target=9494213>,
 <Record source=9502849 weight=17 target=9494213>,
 <Record source=9500501 weight=3 target=9494213>,
 <Record source=9495649 weight=6 target=9494213>,
 <Record source=9495368 weight=12 target=9494213>,
 <Record source=9497555 weight=5 target=9494213>,
 <Record source=9494608 weight=20 target=9494213>,
 <Record source=9494722 weight=10 target=9494213>,
 <Record source=9496475 weight=9 target=9494213>,
 <Record source=9494443 weight=5 target=9494213>,
 <Record source=9497782 weight=5 target=9494213>,
 <Record source=9499910 weight=1 target=9494213>,
 <Record source=9497486 weight=5 target=9494213>]


In [102]:
# Step 2 - Run the algorithm

try:
    query = """
        CALL gds.pageRank.stream('top_influential_reviewers', {relationshipWeightProperty:'weight'})
            
            YIELD nodeId, score
            
            RETURN gds.util.asNode(nodeId).name AS username, score
            ORDER BY score DESC
            LIMIT 10;
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)
    
# yield: In the stream execution mode, the algorithm returns the score for each node. 
# This allows us to inspect the results directly or post-process them in Cypher without any side effects.    
# return: retrieves username based on node ID, and the calculated score

[<Record username='UncleJimbo' score=0.28316456922531064>,
 <Record username='COBEERHUNTER' score=0.27898149412557394>,
 <Record username='Sammy' score=0.17535818328182645>,
 <Record username='StonedTrippin' score=0.1715732288997433>,
 <Record username='metter98' score=0.17126383682053803>,
 <Record username='kojevergas' score=0.16975931325793306>,
 <Record username='emerge077' score=0.16923465799839393>,
 <Record username='biboergosum' score=0.16916817882643637>,
 <Record username='UCLABrewN84' score=0.16905493493162493>,
 <Record username='Knapp85' score=0.16885498206504798>]
